In [ ]:
# Import from meg_qc, relative to the path of this file

# add meg_qc module to the path

from meg_qc.meg_qc_pipeline import make_derivative_meg_qc

config_file_path = 'meg_qc/settings.ini'
raw, raw_cropped_filtered_resampled, QC_derivs, QC_simple, df_head_pos, head_pos, scores_muscle_all1, scores_muscle_all2, scores_muscle_all3, raw1, raw2, raw3, avg_ecg, avg_eog = make_derivative_meg_qc(config_file_path)


In [ ]:
import plotly.graph_objs as go

# Define the data for the first set of points
x1 = [1, 2, 3]
y1 = [4, 5, 6]
z1 = [7, 8, 9]
names1 = ['A', 'B', 'C']

# Define the data for the second set of points
x2 = [1, 2, 3]
y2 = [4, 5, 6]
z2 = [7, 8, 9]
names2 = ['D', 'E', 'F']

# Create the first scatter trace with blue points and names on the right
trace1 = go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers',
    marker=dict(
        color='blue',
        size=10,
        symbol='circle'
    ),
    text=names1,
    hoverinfo='text',
    name='mags'
)

# Create the second scatter trace with red points and names on the left
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(
        color='red',
        size=10,
        symbol='circle'
    ),
    text=names2,
    hoverinfo='text',
    name='grads',
    textposition='top left'  # Set the text position to the left for the second trace
)

# Create the layout for the figure
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Show the figure
fig.show()


In [ ]:
import pickle 
import plotly.graph_objects as go
import numpy as np


# open a file in write binary mode
with open("avg_ecg.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_ecg, f)

with open("avg_eog.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_eog, f)




In [ ]:
import pickle 
import plotly.graph_objects as go
import numpy as np

# open the file in read binary mode
with open("avg_ecg.pkl", "rb") as f:
    # load the list of objects from the file
    my_list = pickle.load(f)

sfreq=1000
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round
fig0=go.Figure
for x in range(0, len(my_list)):
    fig0=my_list[x].plot_epoch_and_peak(t, 'Channels affected by ECG artifact: ', 'mag')
fig0.show()

print(my_list[0].artif_data)

In [ ]:
import plotly.graph_objects as go
import numpy as np
sfreq=1000

fig=go.Figure()
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round

print(t)
for ch in avg_ecg:
    print(t[ch.peak_loc])
    fig=ch.plot_epoch_and_peak(fig, t, 'Channels affected by ECG artifact: ', 'mag')

fig.show()

In [ ]:
import numpy as np
my_array = np.array([1, 2, 3, 4, 5])
print(my_array)
my_array[np.nan]

In [ ]:
#show sensor posiions using mne:

import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/from Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

mne.viz.plot_sensors(raw.info, kind='topomap', ch_type='grad', show_names=True, ch_groups='position')


In [ ]:
#PLOT SENSORS IN 2D with plotly
import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/from Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)


mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}

# Get the sensor locations
sensor_locs = raw.info['chs']
#print(sensor_locs)
#coords_mag=[loc['loc'][:2] for loc in sensor_locs]
coords_mag=[loc['loc'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
#print(len(coords), coords)
print(len(coords_mag), coords_mag)

x = [r[0] for r in coords_mag]
y = [r[1] for r in coords_mag]
#x, y, z = [loc['loc'][:3] for loc in sensor_locs]
names = [loc['ch_name'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
kinds= [loc['kind'] for loc in sensor_locs]
print(kinds)

# Create a scatter plot of the sensor locations
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', text=names))

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000)

# Set the plot title and axis labels
fig.update_layout(title='MEG Sensor Locations', xaxis_title='X', yaxis_title='Y')

# Add a circle shape to the plot to show the position of the head
fig.update_layout(
    shapes=[
        dict(
            type='circle',
            xref='x',
            yref='y',
            x0=-0.1,
            y0=-0.1,
            x1=0.1,
            y1=0.12,
            line=dict(color='red', width=2),
            opacity=0.5
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, -0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, 0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[-0.02, 0.02],
            y0=[0.08, 0.08],
            line=dict(color='black', width=2))
])



# Show the plot 
fig.show()

In [ ]:
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/from Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#PLOT 3 D

def switch_names_on_off(fig):
    # Define the buttons
    buttons = [
    dict(label='Show channel names when hovering',
         method='update',
         args=[{'mode': 'markers'}]),
    dict(label='Always show channel names',
         method='update',
         args=[{'mode': 'markers+text'}])
    ]

    # Add the buttons to the layout
    fig.update_layout(updatemenus=[dict(type='buttons',
                                        showactive=True,
                                        buttons=buttons)])

    return fig



# Extract the sensor locations and names for magnetometers
mag_locs = raw.copy().pick_types(meg='mag').info['chs']
mag_pos = [ch['loc'][:3] for ch in mag_locs]
mag_names = [ch['ch_name'] for ch in mag_locs]

# Create the magnetometer plot with markers only

mag_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in mag_pos],
                                       y=[pos[1] for pos in mag_pos],
                                       z=[pos[2] for pos in mag_pos],
                                       mode='markers',
                                       marker=dict(size=5),
                                       text=mag_names,
                                       hovertemplate='%{text}')],
                                       layout=go.Layout(width=1000, height=1000))

mag_fig.update_layout(title='Magnetometers')

mag_fig = switch_names_on_off(mag_fig)
mag_fig.show()



# Extract the sensor locations and names for gradiometers
grad_locs = raw.copy().pick_types(meg='grad').info['chs']
grad_pos = [ch['loc'][:3] for ch in grad_locs]
grad_names = [ch['ch_name'] for ch in grad_locs]

#since grads have 2 sensors located in the same spot - need to put their names together to make pretty plot labels:

grad_pos_together = []
grad_names_together = []

for i in range(len(grad_pos)-1):
    if all(x == y for x, y in zip(grad_pos[i], grad_pos[i+1])):
        grad_pos_together += [grad_pos[i]]
        grad_names_together += [grad_names[i]+', '+grad_names[i+1]]
    else:
        pass


# Add both sets of gradiometer positions to the plot:
grad_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in grad_pos_together],
                                        y=[pos[1] for pos in grad_pos_together],
                                        z=[pos[2] for pos in grad_pos_together],
                                        mode='markers',
                                        marker=dict(size=5),
                                        text=grad_names_together,
                                        hovertemplate='%{text}')],
                                        layout=go.Layout(width=1000, height=1000))

grad_fig.update_layout(title='Gradiometers')


# Add the button to have names show up on hover or always:
grad_fig = switch_names_on_off(grad_fig)

# Show the plots

grad_fig.show()


In [ ]:

# MUSCLE ARIFACTS IN EMPTYROOM DATA:
# Discussed with Andreas:
# We can see very high muscle scores at the very beginning and end of the empty room recording
# Are these real muscle artifacts or filtering errors?
# Cut out 1st second of the data where they are visible.
# make Fourier transform of the 1st second and see if there are high amplitudes visible for the muscle frequencies - nope 
# most likely this is filtering.
# next, follow the MNE steps for muscle artifact detection: they use first filtering at 11--140 hz, then Hilbert
# plotted raw data after the applied filter, and Hilbert - see cut artifacts in the beginning and end. (WJY arw we sure it  s not hilbert?)
# then, tried to only filter - very noisy data. but most likely the filtering is the source. Because of the cut-off in the beginning and end.
# Solutions: zero padding in the beginning and end before filtering, which will be cut off after. But may still create a jump while filtering and keep the artifact.
# Better: add 2s of dummy data at the beginning and end of the recording, and then crop it out (the data added should be mirrored). This will not create a jump in the filtering.
# Tried

# Problem found! 2 problems: 
# 1st: The main artifact is actually introduced by filtering power lines. filtering the data at 150 Hz (harmonics) clearly creates this artifact.
# Removed that and any other filtering over the range of muscle freqs, since we don't need them anyways. (over 140 Hz)
# 2nd: Still some artifact is present in the beginning and end of the recording. For this attach mirrored data on both ends.
# Then detect muscle, then cut the resulting scores away for the attached period.
# There will be still some very minimal artifact at the beginning/end of this attachment - probably due to the attachment itself: the mirrored data is still not a normally shaped signal.
# See example in cell above of how all 3 option look: oroginal, with attached data and with attached and cut away.

import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/from Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#%matplotlib qt
raw_first = raw.copy().crop(tmin=0, tmax=1)
#raw_first.plot()


data=raw_first.get_data()

window = np.hanning(data.shape[-1])*data

window

freqs = np.fft.rfftfreq(window.shape[-1], 1/raw.info['sfreq'])

components = np.fft.fft(window, axis=-1)

components.shape

fig = go.Figure()
for ch in range(15, 250):
    fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[ch, 0:500])))
fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[0, 0:500])))
#from mne annotate_muscle_zscore:
from scipy.stats import zscore
from scipy.ndimage import label

filter_freq=(110, 140)
ch_type = 'mag'

raw_copy = raw_first.copy()
raw_copy.load_data()

if ch_type is None:
    raw_ch_type = raw_copy.get_channel_types()
    if 'mag' in raw_ch_type:
        ch_type = 'mag'
    elif 'grad' in raw_ch_type:
        ch_type = 'grad'
    elif 'eeg' in raw_ch_type:
        ch_type = 'eeg'
    else:
        raise ValueError('No M/EEG channel types found, please specify a'
                            ' ch_type or provide M/EEG sensor data')
    logger.info('Using %s sensors for muscle artifact detection'
                % (ch_type))

if ch_type in ('mag', 'grad'):
    raw_copy.pick_types(meg=ch_type, ref_meg=False)
else:
    ch_type = {'meg': False, ch_type: True}
    raw_copy.pick_types(**ch_type)

#raw_copy.filter(filter_freq[0], filter_freq[1], fir_design='firwin',
#                pad="reflect_limited")

hilb_applied=raw_copy.apply_hilbert(envelope=True)
hilb_applied.plot()



In [ ]:
import numpy as np

# Load the list of values into a NumPy array
values = np.array([1, 2, 1.8, 2.5, 3, 3.5, 4, 3.8, 5, 4, 2, 2.1, 1, 0, 2, 4, 6])



import numpy as np
from scipy.signal import butter, filtfilt, savgol_filter, find_peaks

import plotly.graph_objects as go
import numpy as np

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

data = np.random.randn(1000) #no wave shape
# Load the noisy wave data into a NumPy array
wave_data = data

# Create a Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=wave_data, mode='lines', name='Noisy Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')
fig.show()

# Apply a low-pass filter to remove high-frequency noise
b, a = butter(5, 0.1, 'low')
filtered_data = filtfilt(b, a, wave_data)

# plot filtered data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=filtered_data, mode='lines', name='Filtered Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Filtered Wave Shape')
fig.show()

# Apply a Savitzky-Golay filter to further reduce noise and extract the underlying wave shape
#smoothed_data = savgol_filter(wave_data, window_length=int(len(wave_data)/4), polyorder=3)
smoothed_data = savgol_filter(data, window_length=100, polyorder=3)

# Identify the shape of the wave using peak detection or curve fitting
# For example, you can use the `scipy.signal.find_peaks` function to detect peaks in the smoothed data
#peaks, _ = find_peaks(smoothed_data, height=0.5*np.max(smoothed_data))
peaks, _ = find_peaks(smoothed_data)

# plot smoothed data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=smoothed_data, mode='lines', name='Smoothed Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Smoothed Wave Shape')
fig.add_trace(go.Scatter(x=t[peaks], y=smoothed_data[peaks], mode='markers', name='Peaks'))
fig.show()



In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

# Find the peaks in the wave
peaks, _ = find_peaks(y)

# Count the number of peaks
num_peaks = len(peaks)

# Create a Plotly figure
fig = go.Figure()

# Add the noisy wave shape to the figure
fig.add_trace(go.Scatter(x=t, y=y, mode='lines', name='Noisy Wave'))

# Add the peaks to the figure
fig.add_trace(go.Scatter(x=t[peaks], y=y[peaks], mode='markers', name='Peaks'))

# Add axis labels and a title
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')

# Show the figure and print the number of peaks
fig.show()
print(f'The wave has {num_peaks} crest(s).')

In [ ]:
import mne
picks_EOG = mne.pick_types(raw.info, eog=True)
eog_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_EOG]
eog_ch_name

In [ ]:
one_psd = [2.93686870e-12, 5.37336497e-13, 2.34749324e-13, 1.70403629e-13
, 1.42868936e-13, 1.10614848e-13, 1.01586902e-13, 9.41699507e-14
, 8.41904711e-14, 7.56254639e-14, 6.98933286e-14, 6.47116338e-14
, 5.37107007e-14, 5.42174045e-14, 4.78692577e-14, 4.36476164e-14
, 3.69272073e-14, 3.81479068e-14, 4.07614720e-14, 3.71683505e-14
, 3.74843265e-14, 3.57210926e-14, 3.99173535e-14, 4.87143053e-14
, 4.20066645e-14, 3.84896719e-14, 3.13482998e-14, 2.86289627e-14
, 2.82586165e-14, 2.71780036e-14, 2.48692250e-14, 2.71251350e-14
, 2.79561808e-14, 2.72047767e-14, 2.79637330e-14, 2.55955578e-14
, 2.53291180e-14, 2.01500680e-14, 2.12080778e-14, 2.18529602e-14
, 2.12775368e-14, 2.14334140e-14, 2.18751322e-14, 1.97884378e-14
, 1.89952388e-14, 1.79404586e-14, 2.01555022e-14, 2.21105500e-14
, 1.87897255e-14, 1.95585625e-14, 2.07067862e-14, 2.15786185e-14
, 1.78539522e-14, 1.89461927e-14, 1.83714513e-14, 1.91272285e-14
, 1.81790918e-14, 1.51935998e-14, 1.55331746e-14, 1.37627320e-14
, 1.37333973e-14, 1.47261781e-14, 1.35323358e-14, 1.23234074e-14
, 1.26296023e-14, 1.39794705e-14, 1.32391885e-14, 1.33172509e-14
, 1.40752537e-14, 1.35291881e-14, 1.46771014e-14, 1.57039580e-14
, 1.76870590e-14, 2.11409680e-14, 2.66470647e-14, 2.09066429e-14
, 1.68226688e-14, 1.63034232e-14, 1.32317697e-14, 1.20372472e-14
, 1.10395275e-14, 1.17336558e-14, 1.12817157e-14, 1.31068881e-14
, 1.36940739e-14, 1.48016686e-14, 1.35999052e-14, 1.56644411e-14
, 1.51726149e-14, 1.95274934e-14, 1.84669709e-14, 1.89443054e-14
, 1.82544652e-14, 1.92617658e-14, 1.80902967e-14, 2.17239287e-14
, 2.67917043e-14, 4.45194367e-14, 2.01857457e-12, 4.40594585e-12
, 1.74602155e-12, 4.30562941e-14, 2.53461498e-14, 1.87278757e-14
, 1.54232144e-14, 1.67924147e-14, 1.22749773e-14, 1.25017017e-14
, 1.22475994e-14, 1.02921463e-14, 1.07700101e-14, 1.02658035e-14
, 9.54949869e-15, 9.84280694e-15, 8.88745310e-15, 9.02206922e-15
, 8.47210049e-15, 8.64491709e-15, 1.32254861e-14, 1.89573615e-14
, 1.23079196e-14, 8.62994931e-15, 8.12535185e-15, 8.01035318e-15
, 7.53220890e-15, 8.02056256e-15, 7.90231409e-15, 7.63270083e-15
, 7.93212379e-15, 7.28368608e-15, 7.59772607e-15, 7.26136429e-15
, 7.86504197e-15, 7.36256360e-15, 7.02847343e-15, 7.08620266e-15
, 6.86068169e-15, 7.21792433e-15, 7.28674098e-15, 6.88181371e-15
, 6.78751472e-15, 6.59002904e-15, 6.74850515e-15, 6.53743454e-15
, 6.70834535e-15, 6.72520433e-15, 6.78371437e-15, 6.70420118e-15
, 6.97442862e-15, 7.26767528e-15, 6.79388360e-15, 6.83101277e-15
, 6.90684197e-15, 6.45716620e-15, 6.66190889e-15, 6.49304182e-15
, 6.38068712e-15, 6.29160702e-15, 5.92354089e-15, 6.33890242e-15
, 6.33787606e-15, 5.76688121e-15, 6.31821916e-15, 6.34536916e-15
, 6.51250512e-15, 6.43164190e-15, 6.46530769e-15, 6.44724883e-15
, 7.48305304e-15, 7.50925230e-15, 6.28419317e-15, 6.33908319e-15
, 5.86954984e-15, 6.54561206e-15, 6.08872456e-15, 6.40874736e-15
, 5.95870142e-15, 6.13488554e-15, 5.83721527e-15, 5.87793931e-15
, 5.81207088e-15, 5.98748087e-15, 5.94551525e-15, 5.75415575e-15
, 5.66968278e-15, 6.11006036e-15, 5.72066372e-15, 5.96629716e-15
, 5.80372053e-15, 5.75583336e-15, 5.84628922e-15, 5.63642362e-15
, 5.34942930e-15, 5.75920960e-15, 6.05337029e-15, 6.61372576e-15
, 7.14210116e-15, 6.94968538e-15, 1.85742697e-14, 3.52090532e-14
, 1.48785528e-14, 6.23577963e-15, 5.66229647e-15, 5.39212323e-15
, 5.32890121e-15, 5.54967559e-15, 5.29485491e-15, 5.65665900e-15
, 5.31337965e-15, 5.30139224e-15, 5.21434237e-15, 5.61739646e-15
, 5.62673191e-15, 5.68441483e-15, 5.43332729e-15, 5.34563989e-15
, 5.69510011e-15, 6.43038706e-15, 5.52069097e-15, 5.22891308e-15
, 5.06513758e-15, 5.15715319e-15, 5.32484298e-15, 5.37071225e-15
, 5.24099974e-15, 5.14413780e-15, 5.05322799e-15, 5.27277366e-15
, 5.17209094e-15, 5.19895605e-15, 5.04662049e-15, 5.13492402e-15
, 5.39573281e-15, 5.13639899e-15, 5.29696474e-15, 5.29749076e-15
, 5.23187737e-15, 5.14179424e-15, 1.44011463e-14, 2.29753019e-14
, 8.85041560e-15, 5.16398069e-15, 5.09075672e-15, 5.06681957e-15
, 5.17284653e-15, 4.99688083e-15, 5.01988585e-15, 5.07952302e-15
, 4.97188381e-15, 5.17733558e-15, 4.97124292e-15, 4.96910679e-15
, 4.96283207e-15, 5.07193856e-15, 4.80712108e-15, 4.97630935e-15
, 4.93727883e-15, 4.84091247e-15, 5.07370238e-15, 4.76459850e-15
, 4.86678392e-15, 5.03907955e-15, 4.91645908e-15, 4.99691785e-15
, 4.81326372e-15, 5.56398292e-15, 5.40286001e-15, 4.91762834e-15
, 4.96042200e-15, 4.86125369e-15, 5.04306529e-15, 4.89229744e-15
, 4.93924928e-15, 4.91889752e-15, 4.92336366e-15, 4.91476256e-15
, 4.90731383e-15, 4.79751988e-15, 4.96181659e-15, 5.04353794e-15]

#plot the data with ploty:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

freqs = [i/2 for i in range(0, 280)]
prominence_lvl_pos = 50
prominence_pos=(max(one_psd) - min(one_psd)) / prominence_lvl_pos
noisy_freqs_indexes, _ = find_peaks(one_psd, prominence=prominence_pos)
noisy_freqs_indexes = [int(i) for i in noisy_freqs_indexes]

for i in range(0,2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=freqs, y=one_psd, name='psd'))
    fig.update_layout(title=' PSD', xaxis_title='Frequency', yaxis_title='Amplitude',
            yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e'))

    fig.add_trace(go.Scatter(x=[freqs[noisy_freqs_indexes[0]]], y=[one_psd[noisy_freqs_indexes[0]]], mode='markers', name='peaks'))

    if i == 0:
        fig.update_yaxes(type="log")

    fig.show()


In [ ]:
df_head_pos


In [ ]:
bool('False')

#convert "false" to boolean:
import ast
t = ast.literal_eval('False')
t
